In [2]:
import csv
import numpy as np 
from sklearn.svm import SVR
import matplotlib.pyplot as plt 
import tensorflow as tf
import pandas as pd

2025-05-12 08:35:53.237467: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 08:35:53.456992: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 08:35:53.569247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747038953.733713    1305 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747038953.822500    1305 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747038954.432493    1305 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
dates = []
openPrice = []
highest = []
lowest = []
closePrice = []
volumes = []

In [4]:
with open('aapl.csv')as file:
    reader = csv.reader(file)
    for date, Open, high, low, close, volume in reader:
        dates.append(date)
        openPrice.append(Open)
        highest.append(high)
        lowest.append(low)
        closePrice.append(close)
        volumes.append(volume.replace(",", ""))

In [8]:
print(type(volumes[0]), volumes[0])

<class 'str'> 51216480


RSI = rsi last_close - today_close
Moving Average = mv
Volume = vol

In [6]:
years = []
monthes = []
days = []

for i in range(len(dates)):
    date_sep = dates[i].split("/")

    year = int(date_sep[2])
    month = int(date_sep[0])
    day = int(date_sep[1])

    years.append(year)
    monthes.append(month)
    days.append(day)


In [7]:
print(type(closePrice[1]))

<class 'str'>


up = positive gap between yesterday close and todays close
down = negative gap between yesterday close and todays close

ave = average 
rs = relative 
rsi = rs index

In [ ]:
ups = []
downs = []
closePrices = []

In [ ]:
def rsi(rsi_term):
    for j in len(closePrice):
        finish = int(closePrice[j])
        closePrices.append(finish)
        if closePrices[j] >= closePrices[j-1]:
            up = closePrices[j] - closePrices[j-1]
            ups.append(up)
            
        else:
            down = closePrices[j-1] - closePrices[j]
            downs.append(down)


    ave_ups = sum(ups[0:rsi_term]) / len(ups[0:rsi_term])        
    ave_downs = sum(downs[0:rsi_term]) / len(downs[0:rsi_term])

    rs = ave_ups / ave_downs
    rsi = 100 - (100 / (rs + 1))
    print(rs, rsi)
    return rs, rsi, closePrices

rsi
moving average
volume

Random Tree

rsi over 70 %  -> too much sell
rsi less 30 % -> too much buy

volume
high -> gained attention from investors

Moving Average 
EMA:… EMA_today = α × price_today + (1 - α) × EMA_yesterday
α（平滑化定数）= 2 / (期間 + 1)

-> SMA: SMA_5（今日） = (今日 + 昨日 + 一昨日 + ... + 5日前) の終値合計 ÷ 5

| 状況              | 解釈                     |
| --------------- | ---------------------- |
| RSIが80、出来高が急増   | 過熱気味。反落の可能性あり。         |
| RSIが20、出来高が急増   | 安値圏で注目されてる → 反発の可能性あり。 |
| RSIが50付近、出来高が低い | 横ばい傾向で方向感なし。様子見。       |


In [ ]:
def SMA(term):
    sma = 0
    for k in range(term - 1):
        sma += closePrices[k]
        sma = 1 / term * sma
    return sma

In [ ]:
def shift_volume(term):
    intvolumes = []
    for n in range(len(volumes)):
        intvolume = int(volume[n])
        intvolumes.append(intvolume)
    
    return intvolumes

rsi check
sma check
ema
intvolumes check

term = may be 5 ~ 10
rsi_term = may be 20 ~

In [59]:
data = {
    "rsi > 70%":[1,1,1,1],
    "rsi < 30%": [1,1,1,1],
    "volume": [1,1,1,1],
    "Moving Average": [1,1,1,1]
}
myvar = pd.DataFrame(data, index = ["NVDA", "AAPL", "GGOL", "TESLA", ])
print(myvar)

       rsi > 70%  rsi < 30%  volume  Moving Average
NVDA           1          1       1               1
AAPL           1          1       1               1
GGOL           1          1       1               1
TESLA          1          1       1               1
